# 最佳实践-百炼官方文档智能助手


## 1. 简介
本文介绍基于百炼平台，构建官方文档助手，结合知识库构建，Assistant API调用，实现全代码创建知识库场景的智能体应用。

### 1.1. 依赖功能
* Assistant API的基础API参数生成能力。
* Assistant API的Function calling能力。

### 1.2. 前期准备
* 已开通百炼服务：[开通阿里云百炼大模型服务产品](https://help.aliyun.com/document_detail/2586399.html?spm=a2c4g.2784257.0.i6)。
* 已创建API_KEY: [获取API-KEY](https://help.aliyun.com/document_detail/2712195.html)。
* 已安装最新版SDK：[安装SDK](https://help.aliyun.com/document_detail/2712193.html)。

## 2. 实现代码

### 2.1. 设置环境变量
安装环境依赖，设置您的API-KEY，替换YOUR_DASHCOPE_API_KEY为您自己的API key

In [1]:
# 安装dashscope SDK。
!pip3 install dashscope

In [2]:
# 通过环境变量设置API-KEY
%env DASHSCOPE_API_KEY=YOUR_API_KEY

env: DASHSCOPE_API_KEY=sk-0MOEpyHAR1


### 2.2. 创建知识库

百炼平台支持管控台页面创建知识索引，以及以代码方式进行创建。
以下代码示例兼容LlamaIndex SDK设计。

In [3]:
!pip install llama-index-core
!pip install llama-index-llms-dashscope
!pip install llama-index-readers-file
!pip install llama-index-indices-managed-dashscope-custom

In [4]:
import os
from llama_index.readers.dashscope.base import DashScopeParse
from llama_index.readers.dashscope.utils import ResultType
from llama_index.indices.managed.dashscope import DashScopeCloudIndex, DashScopeCloudRetriever
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.dashscope import DashScope, DashScopeGenerationModels
from typing import cast
from enum import Enum

In [5]:
def ingest_data(file_folder: str, name: str, category_id="default"):
    # 本地文档上传，并完成文档解析，可以指定百炼数据中心的目录
    parse = DashScopeParse(result_type=ResultType.DASHSCOPE_DOCMIND, category_id=category_id)
    file_extractor = {".pdf": parse, '.doc': parse, '.docx': parse}
    documents = SimpleDirectoryReader(
        file_folder, file_extractor=file_extractor
    ).load_data(num_workers=4)

    # 构建知识索引，完成文档切分、向量化和入库操作
    _ = DashScopeCloudIndex.from_documents(documents, name, verbose=True)
    return documents

In [6]:
class ReturnType(str, Enum):
    STRING = "str"
    NODE = "node"


def retrieve_nodes(query: str, name: str, return_type: ReturnType = ReturnType.STRING):
    """
    知识库检索函数
    :param query: 用户Query 
    :param name: 百炼知识索引名称
    :param return_type: 支持返回str或者LlamaIndex nodes
    :return: 
    """
    def _format(doc_name=None, title=None, content=None, chunk_index=1):
        return f"[{chunk_index}] 【文档名】{doc_name}\n 【标题】{title}\n 【正文】{content}"

    index = DashScopeCloudIndex(name)
    retriever = index.as_retriever(rerank_min_score=0.3)
    retriever = cast(DashScopeCloudRetriever, retriever)
    print(f"Pipeline id {retriever.pipeline_id}", flush=True)
    nodes = retriever.retrieve(query)
    if return_type == ReturnType.STRING:
        context_str = "\n\n".join([_format(
            doc_name=node.metadata["doc_name"],
            title=node.metadata["title"],
            content=node.get_content(),
            chunk_index=index + 1
        ) for index, node in enumerate(nodes)])
        return context_str
    else:
        return nodes


def query_rag(query: str, name: str):
    """
    结合LLM，构造Query Engine，完成RAG完整链路
    :param query: 用户Query 
    :param name: 百炼知识索引名称
    :return: 
    """
    dashscope_llm = DashScope(
        model_name=DashScopeGenerationModels.QWEN_MAX, api_key=os.environ["DASHSCOPE_API_KEY"]
    )
    index = DashScopeCloudIndex(name)
    query_engine = index.as_query_engine(llm=dashscope_llm)
    response = query_engine.query(query)
    return response

In [8]:
index_name = "上海AI峰会-百炼文档助手"

"""Step 1: 百炼知识库数据导入"""
docs = ingest_data(
    "/Users/towardsun/Documents/Project/Bailian/Demo", index_name, 
    category_id="cate_ce28ea58687d4db599d974fe5cd7852010001"
)

2024-06-20 11:20:25,173 DashScopeResponseHandler [INFO] 4376184192 : DashScopeParse upload_lease [URL:https://dashscope.aliyuncs.com/api/v1/datacenter/category/cate_ce28ea58687d4db599d974fe5cd7852010001/upload_lease] request_id: f701eb15-278a-964d-a088-7c4abdaf7661.
2024-06-20 11:20:25,173 llama_index.readers.dashscope.base [INFO] 4376184192 : /Users/towardsun/Documents/Project/Bailian/Demo/百炼快速开始.pdf upload lease result: d838e91774414957b966f2c669e24fed.1718853625389
2024-06-20 11:20:25,173 llama_index.readers.dashscope.domain.lease_domains [INFO] 4376184192 : Start upload /Users/towardsun/Documents/Project/Bailian/Demo/百炼快速开始.pdf.
2024-06-20 11:20:25,179 DashScopeResponseHandler [INFO] 4307944832 : DashScopeParse upload_lease [URL:https://dashscope.aliyuncs.com/api/v1/datacenter/category/cate_ce28ea58687d4db599d974fe5cd7852010001/upload_lease] request_id: 566b2fd4-2aef-9a44-9431-22ca869a35a1.
2024-06-20 11:20:25,180 llama_index.readers.dashscope.base [INFO] 4307944832 : /Users/toward

Starting creating index 上海AI峰会-百炼文档助手, pipeline_id: lc8z6l3x1j
Starting ingestion for index 上海AI峰会-百炼文档助手, ingestion_id: 7662b76b45414d5c85ed23227073768c
Current status: RUNNING
Current status: COMPLETED
ingestion_status COMPLETED
failed_docs: []
Index 上海AI峰会-百炼文档助手 created successfully!


In [9]:
"""Step 2: 在线RA检索"""
question = "请问百炼平台如何部署模型"
resp = retrieve_nodes(question, name=index_name)
print(f"Retrieve result:\nQuestion: {question}\nResponse:\n{resp}")

Pipeline id lc8z6l3x1j
Retrieve result:
Question: 请问百炼平台如何部署模型
Response:
[1] 【文档名】模型调优
 【标题】选择模型版本 超参配置
 【正文】选择模型版本 超参配置
模型调优>模型管理操作指南>选择模型版本>超参配置
企业可以通过参数配置来影响模型调优的过程，从而影响模型调优的效果，不同的参数配置训练的结果不同，一般建议使用默认配置。阿里云百炼模型服务平台 训练新模型 入①)【系统升级】我们将于2024年3月15日14:00:00至3月16日08:00:00进行系统升级,届时模型训练任务可能模型服务 训练方式 超参配置企业可以通过参数配置来影响模型调优的过程,从而影响模型调优的效果,不同我的模型选择模型 的参数配置训练的结果不同,一般建议使用默认配置模型广场 恢复默认训练数据模型体验中心恢复默认训练数据模型体验中心 循环次数② 3● 验证数据Prompt工程学习率② 1e-5模型工具混合训练批次大小② 16模型调优 走超孝甜置模型部署>更多配置于开始训练模型评测训练数据 全参训练 高效训练全谷训练通过全量更新模型参数的方式进行学

[2] 【文档名】百炼快速开始
 【标题】创建应用
 【正文】创建应用
快速开始>创建应用
您可以在百炼平台创建多种大模型应用。以下是四个常见的大模型应用场景：●直接调用模型●模型训练●创建智能体API●创建RAG应用新手引导直接调用模型体验模型 调用模型 prompt可视化、免费地体验通义大模型 一键开通,快速调用大模型服务 通过预置Prompt模板, 或Prompt优化工具提升效果立即体验 立即调用月 立即前往模型训练教程◎上传训练数据 训练模型 评测模型准备并上传SFT训练数据 通过多种方法进行模型微调训练 通过可视化工具进行全自动/半自动的评测立即前往 立即前往 立即前往创建智能体API创建智能体立即前往 立即前往创建智能体API创建智能体 体验智能体 调用智能体编辑Prompt井选择插件 配置智能体的同时随时自由测试 通过Assistant API调用智能体服务立即前往 立即前往 立即调用创建RAG应用教程◎数据对接 创建应用 调用应用上传文档或关联企业数据库 一键创建RAG检索增强服务,并自由调试 通过API/SDK调用RAG服务立即前往注 立即前往一

In [10]:
"""Step 3: 在线RAG完整链路"""
resp = query_rag(question, name=index_name)
print(f"Query result:\nQuestion: {question}\nResponse:\n{resp}")

Query result:
Question: 请问百炼平台如何部署模型
Response:
要在百炼平台上部署模型，请按照以下步骤操作：

1. 在模型管理页面，找到您想要部署的模型。
2. 点击“部署新模型”或者选择“自定义模型”，然后从下拉列表中选取您之前自定义训练的模型名称，继续到下一步。
3. 进入资源配置环节，根据需求选择包月资源或按量付费。例如，选择按量付费，并设置实例数为1，之后点击下一步。
4. 在开始部署界面，仔细检查模型的部署配置详情。
5. 确认无误后，点击“开始部署”按钮。
6. 等待状态变为“运行中”，这表示模型部署已经成功完成。

请注意，部署模型将会产生相应的费用，具体费用依据产品计费标准计算，且一旦模型开始部署（除非部署失败），费用即开始产生。


### 2.3. 创建Assistant API

自定义函数，以下示例包括知识库检索和获取百炼模型训练日志。

In [21]:
import sys
from dashscope import Assistants, Messages, Runs, Threads
from llama_index.indices.managed.dashscope import DashScopeCloudIndex, DashScopeCloudRetriever
from typing import cast
import json
from http import HTTPStatus

In [22]:
def retrieve_nodes(rag_query: str):
    """
    环境变量 INDEX_NAME为管控台知识库索引名称。
    :param rag_query: 知识库查询
    :return:
    """
    index = DashScopeCloudIndex("上海AI峰会-百炼文档助手")
    retriever = index.as_retriever()
    retriever = cast(DashScopeCloudRetriever, retriever)
    print(f"Pipeline id {retriever.pipeline_id}", flush=True)
    nodes = retriever.retrieve(rag_query)
    return "\n\n".join([node.get_content() for node in nodes])


def get_train_info(user_id):
    """
    MOCK接口，进行代码测试；
    :param user_id: 用户ID信息
    :return:
    """
    answer = f"{user_id}"
    model_list = ['qwen-max-sft-v1']
    for model in model_list:
        answer += '\nmodel_name: ' + model + '\ntrain_acc ' + str(86.44) + ' \n training time' + '21 hours 40 mintues'

    return str(answer)


function_mapper = {
    "retrieve_nodes": retrieve_nodes,
    "get_train_info": get_train_info
}

In [23]:
def create_assistant():
    # create assistant with information
    assistant = Assistants.create(
        model="qwen-max",
        name='smart helper',
        description='A tool helper.',
        instructions='You are a helpful assistant. When asked a question, use tools wherever possible.',
        tools=[
            {
                'type': 'quark_search'
            },
            {
                'type': 'function',
                'function': {
                    'name': 'retrieve_nodes',
                    'description': '用于获取与百炼平台相关的信息，包括模型训练、部署，应用搭建、API-KEY等',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'rag_query': {
                                'type': 'str',
                                'description': '百炼相关问题'
                            },
                        },
                        'required': ['rag_query']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': 'get_train_info',
                    'description': '用于获取用户部署在百炼平台上用数据额外训练的模型情况。包含SFT模型的具体情况',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'user_id': {
                                'type': 'str',
                                'description': '用户id'
                            },
                        },
                        'required': ['user_id']
                    }
                }
            }
        ],
    )

    if assistant.status_code != HTTPStatus.OK:
        print('Create Assistant failed, status code: %s, code: %s, message: %s' % (
            assistant.status_code, assistant.code, assistant.message))
        sys.exit()
    else:
        print('Create Assistant success, id: %s' % assistant.id)

    return assistant


def create_message(thread_id, content):
    message = Messages.create(thread_id, content=content)
    if message.status_code != HTTPStatus.OK:
        print('Create Message failed, status code: %s, code: %s, message: %s' % (
            message.status_code, message.code, message.message))
        sys.exit()
    else:
        print('Create Message success, id: %s' % message.id)
    return message


def create_thread():
    thread = Threads.create()
    # check result is success.
    if thread.status_code != HTTPStatus.OK:
        print('Create Thread failed, status code: %s, code: %s, message: %s' % (
            thread.status_code, thread.code, thread.message))
        sys.exit()
    else:
        print('Create Thread success, id: %s' % thread.id)
    return thread


def create_run(thread_id, assistant_id):
    run = Runs.create(thread_id, assistant_id=assistant_id)
    if run.status_code != HTTPStatus.OK:
        print('Create Assistant failed, status code: %s, code: %s, message: %s' % (
            run.status_code, run.code, run.message))
    else:
        print('Create Assistant success, id: %s' % run.id)
    return run

In [24]:
def send_message(assistant, message=''):
    # create a thread.
    thread = create_thread()

    # create a message.
    create_message(thread_id=thread.id, content=message)

    # create run
    response = Runs.create(thread.id, assistant_id=assistant.id, stream=True)
    content_str = ""
    for event, run in response:
        if event == "thread.message.delta":
            content_str += run.delta.content.text.value
            print(content_str, flush=True)
    print(run)

    # wait for run completed or requires_action
    run_status = Runs.wait(run.id, thread_id=thread.id)
    print('插件调用前：')

    if run_status.status == 'failed':
        print('run failed:')
        print(run_status.last_error)

    # if prompt input tool result, submit tool result.
    if run_status.required_action:
        f = run_status.required_action.submit_tool_outputs.tool_calls[0].function
        func_name = f['name']
        param = json.loads(f['arguments'])
        print(f"Function name {func_name}, parameters {param}")
        if func_name in function_mapper:
            output = function_mapper[func_name](**param)
        else:
            output = ""

        tool_outputs = [{
            'tool_call_id': run_status.required_action.submit_tool_outputs.tool_calls[0].id,
            'output': output
        }]

        responses = Runs.submit_tool_outputs(run.id,
                                             thread_id=thread.id,
                                             tool_outputs=tool_outputs,
                                             stream=True)
        content_str = ""
        for event, run in responses:
            if event == "thread.message.delta":
                content_str += run.delta.content.text.value
                print(content_str, flush=True)

        # should wait for run completed
        run_status = Runs.wait(run.id, thread_id=thread.id)

    run_status = Runs.get(run.id, thread_id=thread.id)
    # print(run_status)
    # verify_status_code(run_status)

    # get the thread messages.
    msgs = Messages.list(thread.id)
    # print(msgs)
    # print(json.dumps(msgs, default=lambda o: o.__dict__, sort_keys=True, indent=4))

    print("运行结果:")
    for message in msgs['data'][::-1]:
        print("content: ", message['content'][0]['text']['value'])
    print("\n")

In [25]:
demo_assistant = create_assistant()

Create Assistant success, id: asst_49e99bb3-ea4e-4c72-b03f-b6f5dda9511c


In [26]:
send_message(assistant=demo_assistant, message="请一句话概括爱因斯坦的成就？")

Create Thread success, id: thread_9a95766d-d924-4700-8b2b-4fd9ddf89201
Create Message success, id: message_712e93bd-51a5-4300-b012-7431ce3fec51
爱
爱因
爱因斯坦
爱因斯坦提出了相对论，彻底
爱因斯坦提出了相对论，彻底改变了我们对时间、空间、重
爱因斯坦提出了相对论，彻底改变了我们对时间、空间、重力的理解，奠定了现代物理学的基础。
爱因斯坦提出了相对论，彻底改变了我们对时间、空间、重力的理解，奠定了现代物理学的基础。
{'tools': [{'type': 'quark_search'}, {'function': {'name': 'retrieve_nodes', 'description': '用于获取与百炼平台相关的信息，包括模型训练、部署，应用搭建、API-KEY等', 'parameters': {'type': 'object', 'properties': {'rag_query': {'type': 'str', 'description': '百炼相关问题'}}, 'required': ['rag_query']}}, 'type': 'function'}, {'function': {'name': 'get_train_info', 'description': '用于获取用户部署在百炼平台上用数据额外训练的模型情况。包含SFT模型的具体情况', 'parameters': {'type': 'object', 'properties': {'user_id': {'type': 'str', 'description': '用户id'}}, 'required': ['user_id']}}, 'type': 'function'}], 'required_action': None, 'id': 'run_f3f7c866-34f4-402e-b4fe-5545f0ac57f2', 'object': 'thread.run', 'created_at': 1718854081018, 'thread_id': 'thread_9a95766d-d924-4700-8b2b-4fd9ddf89

In [27]:
send_message(assistant=demo_assistant, message="请问如何获取API KEY？")

Create Thread success, id: thread_bbbbfa4b-4d24-4ab2-b56c-e8e255b2383a
Create Message success, id: message_709d89cc-60bb-41e7-9998-a0a2900a82af
{'tools': [{'type': 'quark_search'}, {'function': {'name': 'retrieve_nodes', 'description': '用于获取与百炼平台相关的信息，包括模型训练、部署，应用搭建、API-KEY等', 'parameters': {'type': 'object', 'properties': {'rag_query': {'type': 'str', 'description': '百炼相关问题'}}, 'required': ['rag_query']}}, 'type': 'function'}, {'function': {'name': 'get_train_info', 'description': '用于获取用户部署在百炼平台上用数据额外训练的模型情况。包含SFT模型的具体情况', 'parameters': {'type': 'object', 'properties': {'user_id': {'type': 'str', 'description': '用户id'}}, 'required': ['user_id']}}, 'type': 'function'}], 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': '', 'function': {'arguments': '{"rag_query": "如何获取百炼平台的API KEY"}', 'name': 'retrieve_nodes', 'output': None}, 'type': 'function'}]}, 'type': 'submit_tool_outputs'}, 'id': 'run_28c2c666-ac40-4026-b0c8-8b776851c1cc', 'object': 'thread.run', 'created_at': 17

In [28]:
send_message(assistant=demo_assistant, message="请问用户ID 114578的用户，模型训练结果如何？")

Create Thread success, id: thread_255d4706-302f-4e78-9811-5eb59dfbfe16
Create Message success, id: message_433597f0-4fd3-491a-9eb8-a14c2e973677
{'tools': [{'type': 'quark_search'}, {'function': {'name': 'retrieve_nodes', 'description': '用于获取与百炼平台相关的信息，包括模型训练、部署，应用搭建、API-KEY等', 'parameters': {'type': 'object', 'properties': {'rag_query': {'type': 'str', 'description': '百炼相关问题'}}, 'required': ['rag_query']}}, 'type': 'function'}, {'function': {'name': 'get_train_info', 'description': '用于获取用户部署在百炼平台上用数据额外训练的模型情况。包含SFT模型的具体情况', 'parameters': {'type': 'object', 'properties': {'user_id': {'type': 'str', 'description': '用户id'}}, 'required': ['user_id']}}, 'type': 'function'}], 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': '', 'function': {'arguments': '{"user_id": "114578"}', 'name': 'get_train_info', 'output': None}, 'type': 'function'}]}, 'type': 'submit_tool_outputs'}, 'id': 'run_ad232b49-f04f-49a7-a62d-89bf568312cc', 'object': 'thread.run', 'created_at': 1718854123221,